In [178]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np

from cleaning_script import NOMINAL, CONTINUOUS, ORDINAL, DISCRETE, unique_by_col, outlier_index, get_outliers

# 1 - Data Cleaning Notebook
---

### Contents

1. [Data Cleaning](../code/data_cleaning.ipynb) [This notebook]
2. [Data Correlation](../code/data_correlation.ipynb)
3. [Data Visualization](../code/visualizations.ipynb)
4. [Regression Model Comparison](../code/regression_model.ipynb)
---

# Data Description



### Descriptive Abstract

The provided data set contains information from the Ames Assessor’s Office used in computing assessed values for individual residential properties sold in Ames, IA from 2006 to 2010. The data broadly falls into four major descriptive categories, `ORDINAL`, `NOMINAL`, `DISCRETE` and `CONTINUOUS`. These four cateogries are comprised of 82 features which include 23 nominal, 23 ordinal, 14 discrete, and 20 continuous variables (and 2 additional observation identifiers).

### Notebook Purpose

This notebook is intended to process the "dirty work" of data science. The provided Ames County information will be parsed and cleaned--we will check for obvious erroneous value entries, missing values and explore the organizational schemas contained within the various features. 


Read in Data

In [179]:
# import data
house_train = pd.read_csv('../datasets/train.csv')
house_test = pd.read_csv('../datasets/test.csv')

# increase column width for greater visibility
pd.set_option('display.max_columns', 100)

Fix Column Names

In [180]:
# make all columns in house_test and house_train lowercase and snake cased
house_train.columns = [x.replace(' ', '_').lower() for x in house_train.columns]
house_test.columns = [x.replace(' ', '_').lower() for x in house_test.columns]

# show first five rows
house_train.head(5)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


# CONTINUOUS Variables
Continuous variables are quanitative and can be any infinite real-valued number. Of the features provided in the Ames County assessor data, the following are the `CONTINUOUS` data types:

- `lot_frontage      `
- `lot_area            `
- `mas_vnr_area       `
- `bsmtfin_sf_1        `
- `bsmtfin_sf_2        `
- `bsmt_unf_sf         `
- `total_bsmt_sf       `
- `1st_flr_sf          `
- `2nd_flr_sf          `
- `gr_liv_area         `
- `garage_area         `
- `wood_deck_sf        `
- `open_porch_sf       `
- `enclosed_porch      `
- `3ssn_porch          `
- `screen_porch        `
- `pool_area           `
- `misc_val `

Display `CONTINUOUS` features, check for missing values

In [181]:
# display continuous columns
display(house_train[CONTINUOUS].head())

# check for missing data
house_train[CONTINUOUS].isnull().sum() # lot_frontage missing a lot of data

,lot_frontage,lot_area,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,2nd_flr_sf,gr_liv_area,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val
0,NaN,13517,289.0,533.0,0.0,192.0,725.0,725,754,1479,475.0,0,44,0,0,0,0,0
1,43.0,11492,132.0,637.0,0.0,276.0,913.0,913,1209,2122,559.0,0,74,0,0,0,0,0
2,68.0,7922,0.0,731.0,0.0,326.0,1057.0,1057,0,1057,246.0,0,52,0,0,0,0,0
3,73.0,9802,0.0,0.0,0.0,384.0,384.0,744,700,1444,400.0,100,0,0,0,0,0,0
4,82.0,14235,0.0,0.0,0.0,676.0,676.0,831,614,1445,484.0,0,59,0,0,0,0,0


lot_frontage      330
lot_area            0
mas_vnr_area       22
bsmtfin_sf_1        1
bsmtfin_sf_2        1
bsmt_unf_sf         1
total_bsmt_sf       1
1st_flr_sf          0
2nd_flr_sf          0
gr_liv_area         0
garage_area         1
wood_deck_sf        0
open_porch_sf       0
enclosed_porch      0
3ssn_porch          0
screen_porch        0
pool_area           0
misc_val            0
dtype: int64

The table presented above displays the number of missing values for each `CONTINUOUS` category. Of the 20 continuous categories, 7 are incomplete.

The magnitude and significance of the missing values is explored in section [insert section]

### Outliers / Erroneous Value Screening

In [182]:
# summary statistics
house_train[CONTINUOUS].describe()

,lot_frontage,lot_area,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,2nd_flr_sf,gr_liv_area,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val
count,1721.000000,2051.000000,2029.000000,2050.000000,2050.000000,2050.000000,2050.000000,2051.000000,2051.000000,2051.000000,2050.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,69.055200,10065.208191,99.695909,442.300488,47.959024,567.728293,1057.987805,1164.488055,329.329108,1499.330083,473.671707,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354
std,23.260653,6742.488909,174.963129,461.204124,165.000901,444.954786,449.410704,396.446923,425.671046,500.447829,215.934561,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985
min,21.000000,1300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58.000000,7500.000000,0.000000,0.000000,0.000000,220.000000,793.000000,879.500000,0.000000,1129.000000,319.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68.000000,9430.000000,0.000000,368.000000,0.000000,474.500000,994.500000,1093.000000,0.000000,1444.000000,480.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,80.000000,11513.500000,161.000000,733.750000,0.000000,811.000000,1318.750000,1405.000000,692.500000,1728.500000,576.000000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,313.000000,159000.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,5095.000000,1862.000000,5642.000000,1418.000000,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000


In [183]:
# return columns and indices of values with a Z-score greater than 10
for col in CONTINUOUS:
    df = pd.DataFrame(get_outliers(house_train, col, outlier_index(house_train, col, 10)[1]))
    if df.empty:
        print(f"{col} has no obvious outliers")
    else:
        print(f"{col} has outliers")
        display(df)

lot_frontage has no obvious outliers
lot_area has outliers


,lot_area
471,159000
694,115149


mas_vnr_area has no obvious outliers
bsmtfin_sf_1 has no obvious outliers
bsmtfin_sf_2 has no obvious outliers
bsmt_unf_sf has no obvious outliers
total_bsmt_sf has no obvious outliers
1st_flr_sf has no obvious outliers
2nd_flr_sf has no obvious outliers
gr_liv_area has no obvious outliers
garage_area has no obvious outliers
wood_deck_sf has outliers


,wood_deck_sf
966,1424


open_porch_sf has no obvious outliers
enclosed_porch has no obvious outliers
3ssn_porch has outliers


,3ssn_porch
892,407
1142,255
1219,508
1577,323
2028,290
2045,304


screen_porch has no obvious outliers
pool_area has outliers


,pool_area
52,519
657,576
761,800
960,480
1130,648
1249,738
1875,561


misc_val has outliers


,misc_val
304,12500
765,8300
1225,6500
1885,17000


The results above show the prescense of a number of values beyond the very high Z-score threshold(10) that was set. However, the columns returned in conjunction with the summary statistics also presented above are helpful. With the exception of `lot_area` all of the features have many null values, thus it is likely that these columns represent something that either is or isn't there. Viewed in this context, a pool area of value of 519 sqft is less likely to be an outlier *within pool areas* but just an outlier in that there is a pool to begin with. Using this process we do not detect any **obvious** erroneous values, just two very large `lot_area` values, which is entirely possible in the context of the dataset. 

# DISCRETE Variables

Discrete variables are quanitative and assums a distinct, non-infinite value. Of the features provided in the Ames County assessor data, the following are the `DISCRETE` data types:

- `year_built`
- `year_remod/add`
- `bsmt_full_bath`
- `bsmt_half_bath`
- `full_bath`
- `half_bath`
- `bedroom_abvgr`
- `kitchen_abvgr`
- `totrms_abvgrd`
- `fireplaces`
- `garage_yr_blt`
- `garage_cars`
- `mo_sold`
- `yr_sold`

### Display `DISCRETE` features, check for missing values

In [184]:
# display discrete data
display(house_train[DISCRETE].head())

# check for missing data
house_train[DISCRETE].isnull().sum() # garage_yr_built missing a lot of data

,year_built,year_remod/add,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,garage_yr_blt,garage_cars,mo_sold,yr_sold
0,1976,2005,0.0,0.0,2,1,3,1,6,0,1976.0,2.0,3,2010
1,1996,1997,1.0,0.0,2,1,4,1,8,1,1997.0,2.0,4,2009
2,1953,2007,1.0,0.0,1,0,3,1,5,0,1953.0,1.0,1,2010
3,2006,2007,0.0,0.0,2,1,3,1,7,0,2007.0,2.0,4,2010
4,1900,1993,0.0,0.0,2,0,3,1,6,0,1957.0,2.0,3,2010


year_built          0
year_remod/add      0
bsmt_full_bath      2
bsmt_half_bath      2
full_bath           0
half_bath           0
bedroom_abvgr       0
kitchen_abvgr       0
totrms_abvgrd       0
fireplaces          0
garage_yr_blt     114
garage_cars         1
mo_sold             0
yr_sold             0
dtype: int64

The table presented above displays the number of missing values for each `DISCRETE` category. Of the 14 continuous categories, 3 are incomplete.

The magnitude and significance of the missing values is explored in section [insert section]

### Outliers / Erroneous Value Screening

In [185]:
house_train[DISCRETE].describe() # garage year built has an entry showing 2207! 

,year_built,year_remod/add,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,garage_yr_blt,garage_cars,mo_sold,yr_sold
count,2051.000000,2051.000000,2049.000000,2049.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,1937.000000,2050.000000,2051.000000,2051.000000
mean,1971.708922,1984.190151,0.427526,0.063446,1.577279,0.371039,2.843491,1.042906,6.435885,0.590931,1978.707796,1.776585,6.219893,2007.775719
std,30.177889,21.036250,0.522673,0.251705,0.549279,0.501043,0.826618,0.209790,1.560225,0.638516,25.441094,0.764537,2.744736,1.312014
min,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1895.000000,0.000000,1.000000,2006.000000
25%,1953.500000,1964.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,4.000000,2007.000000
50%,1974.000000,1993.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,6.000000,2008.000000
75%,2001.000000,2004.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,8.000000,2009.000000
max,2010.000000,2010.000000,3.000000,2.000000,4.000000,2.000000,8.000000,3.000000,15.000000,4.000000,2207.000000,5.000000,12.000000,2010.000000


In [186]:
# return columns and indices of values with a Z-score greater than 10
for col in DISCRETE:
    df = pd.DataFrame(get_outliers(house_train, col, outlier_index(house_train, col, 10)[1]))
    if df.empty:
        print(f"{col} has no obvious outliers")
    else:
        print(f"{col} has outliers")
        display(df)

year_built has no obvious outliers
year_remod/add has no obvious outliers
bsmt_full_bath has no obvious outliers
bsmt_half_bath has no obvious outliers
full_bath has no obvious outliers
half_bath has no obvious outliers
bedroom_abvgr has no obvious outliers
kitchen_abvgr has no obvious outliers
totrms_abvgrd has no obvious outliers
fireplaces has no obvious outliers
garage_yr_blt has no obvious outliers
garage_cars has no obvious outliers
mo_sold has no obvious outliers
yr_sold has no obvious outliers


The all of the columns passed through the Z-score screening function, but three of the categories, `garage_yr_blt`, `bsmt_full_bath` and `bsmt_half_bath` are missing values and the returned DataFrame is just full of Nan's when run through the Z-score screener. When compared to the descriptive statistics table above, we see that the two former features look normal, but `garage_yr_blt` has a max value of 2207, some 200 years away from the sale date of the some. 

It's an obvious typo so we'll fix it below. 

In [187]:
house_train.garage_yr_blt.idxmax() # 1699
house_train.loc[1699]['year_built'] # house was built in 2006 and sold in 2007, I'll just assume 2207 = 2007 and reset. 
house_train['garage_yr_blt'].replace(2207, 2007, inplace=True)

# ORDINAL Variables 

An ordinal variable is similar to a categorical value, but with an inherent order or ranking schema. Of the 23 features provided in the Ames County assessor data, the following are the `ORDINAL` data types:

- `lot_shape`
- `utilities`
- `land_slope`
- `house_style`
- `overall_qual`
- `overall_cond`
- `exter_qual`
- `exter_cond`
- `bsmt_qual`
- `bsmt_cond`
- `bsmt_exposure`
- `bsmtfin_type_1`
- `bsmtfin_type_2`
- `heating_qc`
- `electrical`
- `kitchen_qual`
- `functional`
- `fireplace_qu`
- `garage_finish`
- `garage_qual`
- `garage_cond`
- `paved_drive`
- `pool_qc`
- `fence`

### Display `ORDINANAL` features, check for missing values

In [188]:
display(house_train[ORDINAL])

# check for missing data
house_train[ORDINAL].isnull().sum() # fireplace_qu, pool_qc, fence missing a lot

,lot_shape,utilities,land_slope,house_style,overall_qual,overall_cond,exter_qual,exter_cond,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_type_2,heating_qc,electrical,kitchen_qual,functional,fireplace_qu,garage_finish,garage_qual,garage_cond,paved_drive,pool_qc,fence
0,IR1,AllPub,Gtl,2Story,6,8,Gd,TA,TA,TA,No,GLQ,Unf,Ex,SBrkr,Gd,Typ,NaN,RFn,TA,TA,Y,NaN,NaN
1,IR1,AllPub,Gtl,2Story,7,5,Gd,TA,Gd,TA,No,GLQ,Unf,Ex,SBrkr,Gd,Typ,TA,RFn,TA,TA,Y,NaN,NaN
2,Reg,AllPub,Gtl,1Story,5,7,TA,Gd,TA,TA,No,GLQ,Unf,TA,SBrkr,Gd,Typ,NaN,Unf,TA,TA,Y,NaN,NaN
3,Reg,AllPub,Gtl,2Story,5,5,TA,TA,Gd,TA,No,Unf,Unf,Gd,SBrkr,TA,Typ,NaN,Fin,TA,TA,Y,NaN,NaN
4,IR1,AllPub,Gtl,1.5Fin,6,8,TA,TA,Fa,Gd,No,Unf,Unf,TA,SBrkr,TA,Typ,NaN,Unf,TA,TA,N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,IR1,AllPub,Gtl,1Story,8,5,Gd,TA,Gd,TA,Av,GLQ,Unf,Ex,SBrkr,Gd,Typ,Gd,Fin,TA,TA,Y,NaN,NaN
2047,IR1,AllPub,Gtl,1Story,4,5,TA,TA,TA,TA,No,BLQ,Unf,Ex,SBrkr,TA,Typ,NaN,Unf,TA,TA,Y,NaN,NaN
2048,Reg,AllPub,Gtl,1.5Fin,6,6,TA,TA,TA,TA,No,Unf,Unf,Gd,SBrkr,TA,Typ,TA,Unf,Fa,Fa,Y,NaN,NaN
2049,Reg,AllPub,Gtl,1Story,4,5,TA,TA,TA,TA,No,Rec,LwQ,TA,SBrkr,TA,Typ,Gd,Unf,TA,TA,Y,NaN,NaN


lot_shape            0
utilities            0
land_slope           0
house_style          0
overall_qual         0
overall_cond         0
exter_qual           0
exter_cond           0
bsmt_qual           55
bsmt_cond           55
bsmt_exposure       58
bsmtfin_type_1      55
bsmtfin_type_2      56
heating_qc           0
electrical           0
kitchen_qual         0
functional           0
fireplace_qu      1000
garage_finish      114
garage_qual        114
garage_cond        114
paved_drive          0
pool_qc           2042
fence             1651
dtype: int64

The table above shows that a number of features are missing values. However, as with the outlier analysis on the `CONTINUOUS` variables, the abscence of a variable is often ellucidating. For example, `fireplace_qu` has 1000 missing entries but what this most likely represents is 1000 home did not have a fireplace to evaluate. Similarly, `bsmt_`  and `garage_` categories imply the presence of a basement or garage. We can use this information later to forge boolean flags such as `has_fireplace` as we engineer features. 

In [189]:
house_train['paved_drive'] = [1 if x == 'Y' else 0 for x in house_train.paved_drive]
house_test['paved_drive'] = [1 if x == 'Y' else 0 for x in house_test.paved_drive]

The `paved_drive` category is only 'Y' or 'N' values, so we've converted them to $1$'s and $0$'s above.

### Outliers / Erroneous Value Screening

As the `ORDINAL` categories are not numeric, we cannot screen for outliers. Instead, we look for inconsistent spelling regimes, white space errors or other imperfections that will inhibit the cleaning process further down the line--since it will be neccesarry to convert some of these ranked regimes to numeric equivalents in order to yield predictive power.

In [203]:
dict = {}
for col in house_train[ORDINAL].columns:
    if unique_by_col(ORDINAL, house_train[ORDINAL])[col].dtype == object:
        dict.update({col:unique_by_col(ORDINAL, house_train[ORDINAL])[col]})
dict_df = pd.DataFrame({ k:pd.Series(v) for k, v in dict.items() })

dict_df.T

,0,1,2,3,4,5,6,7
lot_shape,IR1,Reg,IR2,IR3,NaN,NaN,NaN,NaN
utilities,AllPub,NoSeWa,NoSewr,NaN,NaN,NaN,NaN,NaN
land_slope,Gtl,Sev,Mod,NaN,NaN,NaN,NaN,NaN
house_style,2Story,1Story,1.5Fin,SFoyer,SLvl,2.5Unf,2.5Fin,1.5Unf
exter_qual,Gd,TA,Ex,Fa,NaN,NaN,NaN,NaN
exter_cond,TA,Gd,Fa,Ex,Po,NaN,NaN,NaN
bsmt_qual,TA,Gd,Fa,NaN,Ex,Po,NaN,NaN
bsmt_cond,TA,Gd,NaN,Fa,Po,Ex,NaN,NaN
bsmt_exposure,No,Gd,Av,NaN,Mn,NaN,NaN,NaN
bsmtfin_type_1,GLQ,Unf,ALQ,Rec,NaN,BLQ,LwQ,NaN


The above shows no inconsistent spellings for example `Ex` and `EX` but, highlights the repeated use of the `Ex-Po` ranking regime, which we convert to numeric equivalents here [INSERT LINK HERE]

# NOMINAL Variables

A ordinal variable is similar to a categorical value, but with an inherent order or ranking schema. Of the 23 features provided in the Ames County assessor data, the following are the `ORDINAL` data types:

In [ ]:
display(house_train[NOMINAL])

# check for missing data
house_train[NOMINAL].isnull().sum() # alley, misc_feature missing a lot 

,ms_subclass,ms_zoning,street,alley,land_contour,lot_config,neighborhood,condition_1,condition_2,bldg_type,house_style,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,foundation,heating,central_air,garage_type,misc_feature,sale_type
0,60,RL,Pave,NaN,Lvl,CulDSac,Sawyer,RRAe,Norm,1Fam,2Story,Gable,CompShg,HdBoard,Plywood,BrkFace,CBlock,GasA,Y,Attchd,NaN,WD
1,60,RL,Pave,NaN,Lvl,CulDSac,SawyerW,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GasA,Y,Attchd,NaN,WD
2,20,RL,Pave,NaN,Lvl,Inside,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,CBlock,GasA,Y,Detchd,NaN,WD
3,60,RL,Pave,NaN,Lvl,Inside,Timber,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,GasA,Y,BuiltIn,NaN,WD
4,50,RL,Pave,NaN,Lvl,Inside,SawyerW,Norm,Norm,1Fam,1.5Fin,Gable,CompShg,Wd Sdng,Plywood,None,PConc,GasA,Y,Detchd,NaN,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,20,RL,Pave,NaN,HLS,Inside,Timber,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,GasA,Y,Attchd,NaN,WD
2047,30,RL,Pave,NaN,Lvl,Inside,Edwards,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,CBlock,GasA,Y,Detchd,NaN,WD
2048,50,RL,Pave,NaN,Bnk,Inside,Crawfor,Norm,Norm,1Fam,1.5Fin,Gable,CompShg,BrkFace,Stone,None,BrkTil,GasA,Y,Detchd,NaN,WD
2049,20,RL,Pave,NaN,Lvl,Corner,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,None,CBlock,GasA,Y,Attchd,NaN,WD


ms_subclass        0
ms_zoning          0
street             0
alley           1911
land_contour       0
lot_config         0
neighborhood       0
condition_1        0
condition_2        0
bldg_type          0
house_style        0
roof_style         0
roof_matl          0
exterior_1st       0
exterior_2nd       0
mas_vnr_type      22
foundation         0
heating            0
central_air        0
garage_type      113
misc_feature    1986
sale_type          0
dtype: int64

In [ ]:
house_train['central_air'] = [1 if x == 'Y' else 0 for x in house_train.central_air]
house_test['central_air'] = [1 if x == 'Y' else 0 for x in house_test.central_air]

# Check For Erroneous Info
- Inconsistent Spellings in the NOMINAL and ORDINAL categories 
- non-numeric values in the CONTINUOUS and DISCRETE categories

NOMINAL

In [ ]:
vals_1 = unique_by_col(NOMINAL, house_train[NOMINAL])

# Brick Common is spelled, BrkComm, BrkCmn and Brk Cmn in exterior_2, mas_vnr_type
# C (all) and A (Agr) is redundant 

for col in house_train[NOMINAL].columns:
    if vals_1[col].dtype == object:
        print(col, vals_1[col])
        print('*' * 50)

# fix weird spelling/whitespace errors for train set 
house_train.exterior_2nd.replace('Brk Cmn', 'BrkComm', inplace=True)
house_train.mas_vnr_type.replace('BrkCmn', 'BrkComm', inplace=True)
house_train.ms_zoning.replace('C (all)', 'C', inplace=True)
house_train.ms_zoning.replace('A (agr)', 'A', inplace=True)
house_train.ms_zoning.replace('I (all)', 'I',inplace=True)
house_train.sale_type.replace('WD ', 'WD', inplace=True)


# fix weird spelling/whitespace errors for test set
house_test.exterior_2nd.replace('Brk Cmn', 'BrkComm', inplace=True)
house_test.mas_vnr_type.replace('BrkCmn', 'BrkComm', inplace=True)
house_test.ms_zoning.replace('C (all)', 'C', inplace=True)
house_test.ms_zoning.replace('A (agr)', 'A', inplace=True)
house_test.ms_zoning.replace('I (all)', 'I',inplace=True)
house_test.sale_type.replace('WD ', 'WD', inplace=True)

# check for fix
house_train.ms_zoning.unique()

ms_zoning ['RL' 'RM' 'FV' 'C (all)' 'A (agr)' 'RH' 'I (all)']
**************************************************
street ['Pave' 'Grvl']
**************************************************
alley [nan 'Pave' 'Grvl']
**************************************************
land_contour ['Lvl' 'HLS' 'Bnk' 'Low']
**************************************************
lot_config ['CulDSac' 'Inside' 'Corner' 'FR2' 'FR3']
**************************************************
neighborhood ['Sawyer' 'SawyerW' 'NAmes' 'Timber' 'Edwards' 'OldTown' 'BrDale'
 'CollgCr' 'Somerst' 'Mitchel' 'StoneBr' 'NridgHt' 'Gilbert' 'Crawfor'
 'IDOTRR' 'NWAmes' 'Veenker' 'MeadowV' 'SWISU' 'NoRidge' 'ClearCr'
 'Blmngtn' 'BrkSide' 'NPkVill' 'Blueste' 'GrnHill' 'Greens' 'Landmrk']
**************************************************
condition_1 ['RRAe' 'Norm' 'PosA' 'Artery' 'Feedr' 'PosN' 'RRAn' 'RRNe' 'RRNn']
**************************************************
condition_2 ['Norm' 'RRNn' 'Feedr' 'Artery' 'PosA' 'PosN' 'RRAe' 'RRAn']

array(['RL', 'RM', 'FV', 'C', 'A', 'RH', 'I'], dtype=object)

ORDINAL

Looking for mispellings or inconsistencies in this rating regime:

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

DISCRETE

In [ ]:
# cast as all Int64's
house_train[DISCRETE] = house_train[DISCRETE].astype('Int64', copy=False)
house_train[DISCRETE].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   year_built      2051 non-null   Int64
 1   year_remod/add  2051 non-null   Int64
 2   bsmt_full_bath  2049 non-null   Int64
 3   bsmt_half_bath  2049 non-null   Int64
 4   full_bath       2051 non-null   Int64
 5   half_bath       2051 non-null   Int64
 6   bedroom_abvgr   2051 non-null   Int64
 7   kitchen_abvgr   2051 non-null   Int64
 8   totrms_abvgrd   2051 non-null   Int64
 9   fireplaces      2051 non-null   Int64
 10  garage_yr_blt   1937 non-null   Int64
 11  garage_cars     2050 non-null   Int64
 12  mo_sold         2051 non-null   Int64
 13  yr_sold         2051 non-null   Int64
dtypes: Int64(14)
memory usage: 252.5 KB


CONTINUOUS

In [ ]:
vals_4 = unique_by_col(CONTINUOUS, house_train[CONTINUOUS])

house_train[CONTINUOUS].astype('Float64', copy=False)
house_train[CONTINUOUS].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   lot_frontage    1721 non-null   float64
 1   lot_area        2051 non-null   int64  
 2   mas_vnr_area    2029 non-null   float64
 3   bsmtfin_sf_1    2050 non-null   float64
 4   bsmtfin_sf_2    2050 non-null   float64
 5   bsmt_unf_sf     2050 non-null   float64
 6   total_bsmt_sf   2050 non-null   float64
 7   1st_flr_sf      2051 non-null   int64  
 8   2nd_flr_sf      2051 non-null   int64  
 9   gr_liv_area     2051 non-null   int64  
 10  garage_area     2050 non-null   float64
 11  wood_deck_sf    2051 non-null   int64  
 12  open_porch_sf   2051 non-null   int64  
 13  enclosed_porch  2051 non-null   int64  
 14  3ssn_porch      2051 non-null   int64  
 15  screen_porch    2051 non-null   int64  
 16  pool_area       2051 non-null   int64  
 17  misc_val        2051 non-null   i

### Examine Columns that are missing a lot of data

In [ ]:
# missing more than 20 % of their data
missing_the_most = {}
# missing anything
missing_some = {}
# not missing anything
full = []

# iterate through the columns, find any columns with missing data, append to appropriate dict or list
for idx, item in house_train.isnull().sum().items():
    missing_amount = round(item/len(house_train),4)
    if missing_amount > .20:
        missing_the_most.update({idx:missing_amount})
    elif item >= 1:
        missing_some.update({idx:missing_amount})
    else:
        full.append(idx)

### Columns Missing A lot 

In [ ]:
# missing_the_most # all of these categories are things a house might just not have, like a fence or a pool or a fireplace
missing_the_most

{'alley': 0.9317,
 'fireplace_qu': 0.4876,
 'pool_qc': 0.9956,
 'fence': 0.805,
 'misc_feature': 0.9683}

In [ ]:
def impute_binaries_for_missing(df):
    df['alley'].fillna(0, inplace=True)
    df['alley'] = [1 if x != 0 else 0 for x in df['alley']]
    df['fence'].fillna(0, inplace=True)
    df['fence'] = [1 if x != 0 else 0 for x in df['fence']]
    df['fireplace_qu'].fillna(0, inplace = True)
    df['has_fireplace'] = [1 if x != 0 else 0 for x in df['fireplace_qu']]

In [ ]:
impute_binaries_for_missing(house_train)
impute_binaries_for_missing(house_test)

### Columns Missing Some

- Lot Frontage - Linear feet of street connected to property

In [ ]:
missing_some # lot frontage is missing 16% of its data but the rest are pretty O.K.  

{'lot_frontage': 0.1609,
 'mas_vnr_type': 0.0107,
 'mas_vnr_area': 0.0107,
 'bsmt_qual': 0.0268,
 'bsmt_cond': 0.0268,
 'bsmt_exposure': 0.0283,
 'bsmtfin_type_1': 0.0268,
 'bsmtfin_sf_1': 0.0005,
 'bsmtfin_type_2': 0.0273,
 'bsmtfin_sf_2': 0.0005,
 'bsmt_unf_sf': 0.0005,
 'total_bsmt_sf': 0.0005,
 'bsmt_full_bath': 0.001,
 'bsmt_half_bath': 0.001,
 'garage_type': 0.0551,
 'garage_yr_blt': 0.0556,
 'garage_finish': 0.0556,
 'garage_cars': 0.0005,
 'garage_area': 0.0005,
 'garage_qual': 0.0556,
 'garage_cond': 0.0556}

# Drop Columns Missing a lot of Data
- We'll deal with the columns missing a small amount of data as it becomes neccesarry in the correlation notebook

In [ ]:
# train
drop = ['fireplace_qu', 'pool_qc', 'misc_feature'] 
house_train.drop(drop, axis=1, inplace=True)
# test
house_test.drop(drop, axis = 1, inplace = True)

# Clean Data to .csv

In [ ]:
# train
house_train.to_csv('../datasets/house_train_clean.csv', index=False)

# test
house_test.to_csv('../datasets/house_test_clean.csv', index = False)

In [ ]:
house_train_clean = pd.read_csv('../datasets/house_train_clean.csv')

In [ ]:
house_test.shape

(878, 78)

In [ ]:
house_train.shape

(2051, 79)